In [1]:
import lionagi as li
from pydantic import Field
import asyncio

In [2]:
"""
- plan code
- write code
- review code

"""

'\n- plan code\n- write code\n- review code\n\n'

In [3]:
class CoderForm(li.Form):
    """
    A form template for developing software project
    """

    template_name: str = "coder template"

    context: str | None= Field(
        None,
        title="Context",
        description="The context for the current issue to achieve",
    )

    plan: str | None = Field(
        None,
        title="Plan",
        description=" breaking down the issue to multi-step plan",
    )

    code: str | None = Field(
        None,
        title="Code",
        description="codes generated by workflow",
    )

    review: str | None = Field(
        None,
        title="review",
        description="comments on generated code"
    )


In [4]:
GUIDANCE_RESPONSE = """
    Guidance from super intelligent code bot:
    {guidance_response}
    Please generate Python functions that satisfies the prompt and follows the provided guidance, while adhering to these coding standards:
    - Use descriptive and meaningful names for variables, functions, and classes.
    - Follow the naming conventions: lowercase with underscores for functions and variables, CamelCase for classes.
    - Keep functions small and focused, doing one thing well.
    - Use 4 spaces for indentation, and avoid mixing spaces and tabs.
    - Limit line length to 79 characters for better readability.
    - Use docstrings to document functions, classes, and modules, describing their purpose, parameters, and return values.
    - Use comments sparingly, and prefer descriptive names and clear code structure over comments.
    - Handle exceptions appropriately and raise exceptions with clear error messages.
    - Use blank lines to separate logical sections of code, but avoid excessive blank lines.
    - Import modules in a specific order: standard library, third-party, and local imports, separated by blank lines.
    - Use consistent quotes (single or double) for strings throughout the codebase.
    - Follow the PEP 8 style guide for more detailed coding standards and best practices.
"""

PLAN_PROMPT = "Please design coding instructions for the following prompt and provide guidance for the coder to follow."
WRITE_PROMPT = "Please write a Python function that satisfies the prompt and follows the provided guidance."
REVIEW_PROMPT = "Please review the following code and remove any unnecessary markdown or descriptions:\n\n{code}\n"

CODER_PROMPTS = {
    "system": GUIDANCE_RESPONSE,
    "plan_code": PLAN_PROMPT,
    "write_code": WRITE_PROMPT,
    "review_code": REVIEW_PROMPT,
}

In [5]:
class Coder(li.Worker):
    name = "coder"
    form_template = CoderForm

    def __init__(self, session=None, system=None, session_kwargs=None):
        super().__init__()
        self.session = session or self._create_session(system, session_kwargs)
        self.tasks = {}
    
    def _create_session(self, system=None, session_kwargs=None):
        session_kwargs = session_kwargs or {}
        session = li.Session(system=system, **session_kwargs)
        return session
    
    def create_new_issue(self, issue):
        if self.forms.get(self.session.branches[0].ln_id):
            new_branch = self.session.new_branch()
            self.forms[new_branch.ln_id] = self.form_template(context = issue)
        else:
            new_branch = self.session.default_branch
            self.forms[new_branch.ln_id] = self.form_template(context = issue)
            self.default_form = self.forms[new_branch.ln_id]

    async def _update_form(self, work):
        if work.status == "COMPLETED":
            for field in work.result.requested_fields:
                value = getattr(work.result, field)
                setattr(work.result.origin, field, value)

    # @li.work:
    #       assignment: the expected input and output fields at this step, e.g. "input1, input2 -> output"
    #                   note: - all input fields should be filled before the step execution
    #                         - check all available forms in the worker at self.forms
    #       form_param_key: the function parameter that is used as the key to map the corresponding form in self.forms
    #                       this feature helps you create a form based the assignment and matched form internally
    #                       e.g. in this coder example, we use branch ln_id for form mapping, 
    #                            as each branch solves one coding issue, and each issue corresponds to one form
    #                       note: - the key can be any python supported dict key types or LionIDable objects
    #       guidance: the task of the work
    #                 note: if guidance is not provided explicitly, the docstring of the function will be used as the guidance if available
    #       capacity: The capacity of the work queue for each batch processing

    @li.work(assignment="context -> plan", form_param_key="branch", guidance=CODER_PROMPTS["plan_code"], capacity=2)
    async def _plan_code(self, branch=None, **kwargs):
        if branch is None:
            branch = self.session.default_branch
        else:
            branch = self.session.branches[branch]
        print(f"*****\nPlanning code for issue: \n----------\n{self.forms[branch.ln_id].context}\n----------\n**")
        result = await branch.chat(**kwargs)
        print(f"*****\nCode plan generated for issue: \n----------\n{self.forms[branch.ln_id].context}\n----------\n**")
        return result
    
    @li.work(assignment="context, plan -> code", form_param_key="branch", guidance=CODER_PROMPTS["write_code"], capacity=2)
    async def _write_code(self, branch=None, **kwargs):
        if branch is None:
            branch = self.session.default_branch
        else:
            branch = self.session.branches[branch]
        print(f"*****\nWriting code for issue: \n----------\n{self.forms[branch.ln_id].context}\n----------\n**")
        result = await branch.chat(**kwargs)
        print(f"*****\nCode written for issue: \n----------\n{self.forms[branch.ln_id].context}\n----------\n**")
        return result
    
    @li.work(assignment="code -> review", form_param_key="branch", guidance=CODER_PROMPTS["review_code"], capacity=2)
    async def _review_code(self, branch=None, **kwargs):
        if branch is None:
            branch = self.session.default_branch
        else:
            branch = self.session.branches[branch]
        print(f"*****\nReviewing code for issue: \n----------\n{self.forms[branch.ln_id].context}\n----------\n**")
        result = await branch.chat(**kwargs)
        print(f"*****\nCode reviewed for issue: \n----------\n{self.forms[branch.ln_id].context}\n----------\n**")
        return result
    
    # handle function scheduling for execute: schedule next work
    async def handle_task(self, branch, work=None, refresh_time=1):
        if work is None:
            task = await self._plan_code(branch=branch)
            if not self.work_functions["_plan_code"].worklog.queue.execution_mode:
                asyncio.create_task(self.work_functions["_plan_code"].worklog.queue.execute(refresh_time=refresh_time))
            return task
        if work.status == "COMPLETED":
            await self._update_form(work)
            if work.async_task_name == "_plan_code":
                task = await self._write_code(branch=branch)
                if not self.work_functions["_write_code"].worklog.queue.execution_mode:
                    asyncio.create_task(self.work_functions["_write_code"].worklog.queue.execute(refresh_time=refresh_time))
                return task
            elif work.async_task_name == "_write_code":
                task = await self._review_code(branch=branch)
                if not self.work_functions["_review_code"].worklog.queue.execution_mode:
                    asyncio.create_task(self.work_functions["_review_code"].worklog.queue.execute(refresh_time=refresh_time))
                return task
            elif work.async_task_name == "_review_code":
                return "COMPLETE"
        elif work.status == "FAILED":
            if branch is None:
                branch = self.session.default_branch
            failed_works = getattr(self, failed_works, [])
            failed_works.append((self.session.branches[branch], work))
            return "FAILED"
    
    # execute the whole process automatically and concurrently
    async def execute(self, refresh_time=1):
        for branch in self.session.branches:
            task = await self.handle_task(branch, refresh_time=refresh_time)
            if task is not None:
                self.tasks[branch.ln_id] = task
        
        while self.tasks:
            for work_func in self.work_functions.values():
                await work_func.worklog.forward()
        
            await asyncio.sleep(refresh_time)

            for branch_id in list(self.tasks.keys()):
                task = await self.handle_task(branch=branch_id, work=self.tasks[branch_id], refresh_time=refresh_time)
                if task == "COMPLETE" or task == "FAILED":
                    self.tasks.pop(branch_id)
                elif task is not None:
                    self.tasks[branch_id] = task
        
        await self.stop()

# Create Coder

In [6]:
coder = Coder(system=CODER_PROMPTS['system'])

### Create Coding Questions

In [7]:
coder.create_new_issue("write a pure python function that takes a list of integers and returns the sum of all the integers in the list.")
coder.create_new_issue("write a pure python function that takes a list of integers and returns the maximum integer in the list.")
coder.create_new_issue("Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers from the list and the values are the counts of how many times each integer appears in the list.")

# Execute Step-by-Step

## Plan code given context

### Schedule Function

with `@li.work`, calling the function will only parse it to a `work` and add it to the pending list of the internal `WorkLog`. 

In [8]:
for branch in coder.session.branches:
    await coder._plan_code(branch=branch.ln_id)
plan_log = coder.work_functions["_plan_code"].worklog

In [9]:
plan_log.pending

Progression(['b6f66d964148921ff79052f28f9d8ab5', '6bef818534549d005a594bb17ebb85f9', '7a98045c64094940aa779792263af62e'])

In [10]:
plan_log.pile

                              ln_id                     created metadata  \
0  b6f66d964148921ff79052f28f9d8ab5  2024-06-03T22:59:34.630140       {}   
1  6bef818534549d005a594bb17ebb85f9  2024-06-03T22:59:34.630249       {}   
2  7a98045c64094940aa779792263af62e  2024-06-03T22:59:34.630328       {}   

  content embedding              status result error  \
0    None        []  WorkStatus.PENDING   None  None   
1    None        []  WorkStatus.PENDING   None  None   
2    None        []  WorkStatus.PENDING   None  None   

                                          async_task async_task_name  \
0  <coroutine object WorkFunction.perform at 0x12...      _plan_code   
1  <coroutine object WorkFunction.perform at 0x12...      _plan_code   
2  <coroutine object WorkFunction.perform at 0x12...      _plan_code   

  completion_timestamp duration lion_class  
0                 None     None       Work  
1                 None     None       Work  
2                 None     None       Work  

`worklog.forward()` will forward all pending `work`s to the internal `WorkQueue`, which manages `work` executions

In [11]:
await plan_log.forward() # forward to WorkQueue and wait for processing

`workqueue.process()` will execute functions in queue by batch. Batch size is equal to the capacity defined in `@li.work`

In [12]:
print("first batc:")
await plan_log.queue.process() # process first two functions in queue

first batc:
*****
Planning code for issue: 
----------
write a pure python function that takes a list of integers and returns the sum of all the integers in the list.
----------
**
*****
Planning code for issue: 
----------
write a pure python function that takes a list of integers and returns the maximum integer in the list.
----------
**
*****
Code plan generated for issue: 
----------
write a pure python function that takes a list of integers and returns the maximum integer in the list.
----------
**
*****
Code plan generated for issue: 
----------
write a pure python function that takes a list of integers and returns the sum of all the integers in the list.
----------
**


In [13]:
print("secend batch:")
await plan_log.queue.process()

secend batch:
*****
Planning code for issue: 
----------
Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers from the list and the values are the counts of how many times each integer appears in the list.
----------
**
*****
Code plan generated for issue: 
----------
Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers from the list and the values are the counts of how many times each integer appears in the list.
----------
**


In [14]:
# update the step result
for work in plan_log.pile:
    await coder._update_form(work)

## Write code given context and plan

In [15]:
for branch in coder.session.branches:
    await coder._write_code(branch=branch)
write_log = coder.work_functions["_write_code"].worklog

In [16]:
await write_log.forward()
print("first batch:")
await write_log.queue.process()
print("secend batch:")
await write_log.queue.process()

first batch:
*****
Writing code for issue: 
----------
write a pure python function that takes a list of integers and returns the sum of all the integers in the list.
----------
**
*****
Writing code for issue: 
----------
write a pure python function that takes a list of integers and returns the maximum integer in the list.
----------
**
*****
Code written for issue: 
----------
write a pure python function that takes a list of integers and returns the sum of all the integers in the list.
----------
**
*****
Code written for issue: 
----------
write a pure python function that takes a list of integers and returns the maximum integer in the list.
----------
**
secend batch:
*****
Writing code for issue: 
----------
Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers from the list and the values are the counts of how many times each integer appears in the list.
----------
**
*****
Code written for issue: 
----------
Write a

In [17]:
for work in write_log.pile:
    await coder._update_form(work)

## Review code given code

In [18]:
for branch in coder.session.branches:
    await coder._review_code(branch=branch)
review_log = coder.work_functions["_review_code"].worklog

In [19]:
await review_log.forward()
print("first batch:")
await review_log.queue.process()
print("secend batch:")
await review_log.queue.process()

first batch:
*****
Reviewing code for issue: 
----------
write a pure python function that takes a list of integers and returns the sum of all the integers in the list.
----------
**
*****
Reviewing code for issue: 
----------
write a pure python function that takes a list of integers and returns the maximum integer in the list.
----------
**
*****
Code reviewed for issue: 
----------
write a pure python function that takes a list of integers and returns the sum of all the integers in the list.
----------
**
*****
Code reviewed for issue: 
----------
write a pure python function that takes a list of integers and returns the maximum integer in the list.
----------
**
secend batch:
*****
Reviewing code for issue: 
----------
Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers from the list and the values are the counts of how many times each integer appears in the list.
----------
**
*****
Code reviewed for issue: 
---------

In [20]:
for work in review_log.pile:
    await coder._update_form(work)

# Results

In [21]:
from IPython.display import Markdown, display

for k in coder.forms.keys():
    display(Markdown("Context: " + coder.forms[k].context))
    display(Markdown("Plan: " + coder.forms[k].plan))
    display(Markdown("```python \n" + coder.forms[k].code + "\n```"))
    display(Markdown(coder.forms[k].review))
    print("*************************************************************************************************")

Context: write a pure python function that takes a list of integers and returns the sum of all the integers in the list.

Plan: {"step": "Define the function signature", "description": "Create a function named 'sum_of_integers' that takes a single parameter 'numbers', which is expected to be a list of integers."}, {"step": "Initialize a variable to store the sum", "description": "Inside the function, initialize a variable 'total' to 0. This will be used to accumulate the sum of the integers."}, {"step": "Iterate through the list", "description": "Use a for loop to iterate over each integer in the 'numbers' list."}, {"step": "Add each integer to the total", "description": "Within the loop, add the current integer to the 'total' variable."}, {"step": "Return the total sum", "description": "After the loop completes, return the 'total' variable, which now contains the sum of all integers in the list."}

```python 
def sum_of_integers(numbers):
    """
    Calculate the sum of all integers in a list.

    Parameters:
    numbers (list of int): A list of integers to be summed.

    Returns:
    int: The sum of all integers in the list.
    """
    total = 0
    for number in numbers:
        total += number
    return total

```

The provided code is well-structured and follows the given plan accurately. The docstring is useful for understanding the function's purpose, parameters, and return value. There are no unnecessary markdown or descriptions in the code. The function is concise and adheres to the coding standards provided. No changes are necessary.

*************************************************************************************************


...

Context: Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers from the list and the values are the counts of how many times each integer appears in the list.

Plan: {"step": "Define the function signature", "details": "Create a function named 'count_integers' that takes a single parameter 'int_list', which is expected to be a list of integers."}, {"step": "Initialize an empty dictionary", "details": "Within the function, initialize an empty dictionary named 'counts' to store the counts of each integer."}, {"step": "Iterate through the list", "details": "Use a for loop to iterate over each integer in 'int_list'."}, {"step": "Update the dictionary", "details": "For each integer in the list, check if it is already a key in the 'counts' dictionary. If it is, increment its value by 1. If it is not, add it to the dictionary with a value of 1."}, {"step": "Return the dictionary", "details": "After the loop completes, return the 'counts' dictionary."}

```python 
def count_integers(int_list):
    """
    Count the occurrences of each integer in the given list.

    Parameters:
    int_list (list of int): A list of integers.

    Returns:
    dict: A dictionary where the keys are integers from the list and the values are the counts of how many times each integer appears in the list.
    """
    counts = {}

    for integer in int_list:
        if integer in counts:
            counts[integer] += 1
        else:
            counts[integer] = 1

    return counts
```

The provided code is well-structured and follows the given plan accurately. The function 'count_integers' is clearly defined, and the docstring provides a good description of the function's purpose, parameters, and return value. The code is concise and performs the task as expected. No unnecessary markdown or descriptions are present in the code. Overall, the code is clean and adheres to the provided guidance.

*************************************************************************************************


# Execute Automatically and Concurrently

In [22]:
coder_auto = Coder()

In [23]:
coder_auto.create_new_issue("write a pure python function that takes a list of integers and returns the sum of all the integers in the list.")
coder_auto.create_new_issue("write a pure python function that takes a list of integers and returns the maximum integer in the list.")
coder_auto.create_new_issue("Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers from the list and the values are the counts of how many times each integer appears in the list.")

In [24]:
await coder_auto.execute(refresh_time=1)

*****
Planning code for issue: 
----------
write a pure python function that takes a list of integers and returns the sum of all the integers in the list.
----------
**
*****
Planning code for issue: 
----------
write a pure python function that takes a list of integers and returns the maximum integer in the list.
----------
**
*****
Code plan generated for issue: 
----------
write a pure python function that takes a list of integers and returns the sum of all the integers in the list.
----------
**
*****
Writing code for issue: 
----------
write a pure python function that takes a list of integers and returns the sum of all the integers in the list.
----------
**
*****
Code plan generated for issue: 
----------
write a pure python function that takes a list of integers and returns the maximum integer in the list.
----------
**
*****
Planning code for issue: 
----------
Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers f

In [25]:
from IPython.display import Markdown, display

for k in coder.forms.keys():
    display(Markdown("Context: " + coder.forms[k].context))
    display(Markdown("Plan: " + coder.forms[k].plan))
    display(Markdown("```python \n" + coder.forms[k].code + "\n```"))
    display(Markdown(coder.forms[k].review))
    print("*************************************************************************************************")

Context: write a pure python function that takes a list of integers and returns the sum of all the integers in the list.

Plan: {"step": "Define the function signature", "description": "Create a function named 'sum_of_integers' that takes a single parameter 'numbers', which is expected to be a list of integers."}, {"step": "Initialize a variable to store the sum", "description": "Inside the function, initialize a variable 'total' to 0. This will be used to accumulate the sum of the integers."}, {"step": "Iterate through the list", "description": "Use a for loop to iterate over each integer in the 'numbers' list."}, {"step": "Add each integer to the total", "description": "Within the loop, add the current integer to the 'total' variable."}, {"step": "Return the total sum", "description": "After the loop completes, return the 'total' variable, which now contains the sum of all integers in the list."}

```python 
def sum_of_integers(numbers):
    """
    Calculate the sum of all integers in a list.

    Parameters:
    numbers (list of int): A list of integers to be summed.

    Returns:
    int: The sum of all integers in the list.
    """
    total = 0
    for number in numbers:
        total += number
    return total

```

The provided code is well-structured and follows the given plan accurately. The docstring is useful for understanding the function's purpose, parameters, and return value. There are no unnecessary markdown or descriptions in the code. The function is concise and adheres to the coding standards provided. No changes are necessary.

*************************************************************************************************


...

Context: Write a pure python function that takes a list of integers and returns a dictionary where the keys are the integers from the list and the values are the counts of how many times each integer appears in the list.

Plan: {"step": "Define the function signature", "details": "Create a function named 'count_integers' that takes a single parameter 'int_list', which is expected to be a list of integers."}, {"step": "Initialize an empty dictionary", "details": "Within the function, initialize an empty dictionary named 'counts' to store the counts of each integer."}, {"step": "Iterate through the list", "details": "Use a for loop to iterate over each integer in 'int_list'."}, {"step": "Update the dictionary", "details": "For each integer in the list, check if it is already a key in the 'counts' dictionary. If it is, increment its value by 1. If it is not, add it to the dictionary with a value of 1."}, {"step": "Return the dictionary", "details": "After the loop completes, return the 'counts' dictionary."}

```python 
def count_integers(int_list):
    """
    Count the occurrences of each integer in the given list.

    Parameters:
    int_list (list of int): A list of integers.

    Returns:
    dict: A dictionary where the keys are integers from the list and the values are the counts of how many times each integer appears in the list.
    """
    counts = {}

    for integer in int_list:
        if integer in counts:
            counts[integer] += 1
        else:
            counts[integer] = 1

    return counts
```

The provided code is well-structured and follows the given plan accurately. The function 'count_integers' is clearly defined, and the docstring provides a good description of the function's purpose, parameters, and return value. The code is concise and performs the task as expected. No unnecessary markdown or descriptions are present in the code. Overall, the code is clean and adheres to the provided guidance.

*************************************************************************************************
